In [ ]:
#
#
# 1 ~ 4
#
#

In [ ]:
# 5-1 기본 실습

# SimpleRNN (기본 순환 구조)
'''
숫자 시퀀스 예측
입력: [1, 2, 3] → 출력: 4
입력: [2, 3, 4] → 출력: 5
입력: [3, 4, 5] → 출력: 6
'''

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, SimpleRNN, Dense

# 시계열 데이터처럼 연속된 숫자를 기반으로 다음 값을 예측하는 문제

# 1. 데이터 준비
# 입력 시퀀스 (4개 샘플, 각 샘플은 길이 3인 시퀀스)
# 정답 (각 시퀀스의 다음 숫자)
def create_data(i):
  train_list = []
  val_list = []
  for j in range(1, i+1):
    train = [j, j+1, j+2]
    train_list.append(train)
    val = j+3
    val_list.append(val)
  train_list = np.array(train_list)
  val_list = np.array(val_list)
  return train_list, val_list
X, y = create_data(4)

# RNN 입력형태로 reshape: (샘플 수, 타임스텝 수, 특성 수)
# 숫자 하나씩 보므로 특성(feature)은 1
X = X.reshape((-1, 3, 1))

# 2. 모델 구성
model = Sequential()
# Input 레이어로 입력 형태를 명시 (3타임스텝, 1특성)
# SimpleRNN 레이어 (유닛 10개, 활성화함수: relu)
# Dense 출력층 (출력 노드 1개: 다음 숫자 1개 예측)
model.add(Input(shape=(3, 1)))
model.add(SimpleRNN(32, activation='relu'))
model.add(Dense(1))

# 3. 모델 컴파일
# optimizer = 'adam': 최적화 알고리즘
# loss = 'mse': 회귀 문제이므로 평균제곱오차 사용
model.compile(
  optimizer = 'adam',
  loss = 'mse',
  metrics = ['mae']
)

# 4. 모델 학습
# epochs = 200: 반복 횟수
# verbose = 0: 진행 상황 출력 생략(0)
model.fit(
  X, y,
  epochs = 200,
  verbose = 1
)

# 5. 예측
# 예측 대상 시퀀스: [5, 6, 7]
# 샘플 수 차원 추가 필요
test_input = np.array([5, 6, 7]).reshape((1, 3, 1))

# 예측 수행
predicted = model.predict(test_input, verbose=0)
print("예측결과:", predicted[0][0])